In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive
!unzip BrixIA_small.zip -d /content/
%cd /content/gdrive/MyDrive/Colab\ Notebooks

In [3]:
import torch
from torchvision import transforms
import cxr_dataset as CXR

import eval_model as E

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PATH_TO_IMAGES = "/content/BrixIAsmall"
MODEL_PATH = '/content/weighted_classification_checkpoint_best'

In [11]:
# use imagenet mean,std for normalization
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

# create dataloader
dataset = CXR.CXRDataset(
    path_to_images=PATH_TO_IMAGES,
    fold='test',
    transform=data_transforms['val'],
    fine_tune=True)
dataloader = torch.utils.data.DataLoader(
    dataset, 32, shuffle=False, num_workers=8)

saved_model = torch.load(MODEL_PATH)
model = saved_model['model']
model.to(device)
del saved_model

_, auc = E.make_pred_multilabel(dataloader, model, fine_tune=True)
print(auc)

     label       auc        AP
0    Covid  0.863185  0.981263
1  NoCovid  0.849953  0.460977
